Import necessary libraries

In [11]:
# Import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import display, Image
import matplotlib.pyplot as plt
from google.colab import output
import io
from PIL import Image as PILImage
import base64

# Load the pre-trained MNIST model
model = load_model('/content/mnist_model.keras')

# Helper function to preprocess the image into model's input shape
def to_model_shape(x28):
    x28 = x28.astype("float32") / 255.0
    return x28.reshape(1, 28, 28, 1)

def preprocess_mnist(roi_bgr):
    """Convert the ROI to a 28x28 image and preprocess it for model prediction"""
    # Convert to grayscale and apply blur + threshold
    g = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(g, (5, 5), 0)
    th = cv2.adaptiveThreshold(
        g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )

    # Find contours and select the largest one
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None, {"reason": "no contour"}

    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    if w * h < 300:   # Reject tiny contours
        return None, {"reason": "small contour"}

    # Prepare the region of interest (ROI) for prediction
    roi = th[y:y+h, x:x+w]
    resized_roi = cv2.resize(roi, (28, 28))
    model_input = to_model_shape(resized_roi)
    return model_input, {"bounding_box": (x, y, w, h)}

# JavaScript function to capture an image using the webcam
from google.colab.output import eval_js
def take_picture():
    js = """
    async function takePicture() {
        const video = document.createElement('video');
        video.width = 640;
        video.height = 480;
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        await new Promise((resolve) => { video.onloadedmetadata = () => { resolve(video); }; });
        video.play();
        const canvas = document.createElement('canvas');
        canvas.width = 640;
        canvas.height = 480;
        const context = canvas.getContext('2d');
        context.drawImage(video, 0, 0, 640, 480);
        const imageUrl = canvas.toDataURL('image/png');
        stream.getTracks().forEach(track => track.stop());
        return imageUrl;
    }
    takePicture();
    """
    return eval_js(js)

# Capture and predict number
image_data = take_picture()  # Capture image from webcam
image_bytes = io.BytesIO(base64.b64decode(image_data.split(',')[1]))  # Decode the image data
img = PILImage.open(image_bytes)  # Convert to PIL image
plt.imshow(img)  # Display the image
plt.show()

# Convert the image to numpy array (OpenCV format)
roi_bgr = np.array(img)

# Preprocess the captured image for prediction
model_input, _ = preprocess_mnist(roi_bgr)

# Predict the digit using the model
prediction = model.predict(model_input)
predicted_digit = np.argmax(prediction)

# Output the predicted digit
print(f"Predicted Digit: {predicted_digit}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicted Digit: 8
